In [1]:
import sys

sys.path.append("../..")

#Creation of the subsets and hazard rates

In [2]:
indiv_subsets = [frozenset([i]) for i in range(3)]
groups_subsets = [frozenset([0, 1]), frozenset([0, 1, 2])]

subsets = indiv_subsets+groups_subsets
print subsets

[frozenset([0]), frozenset([1]), frozenset([2]), frozenset([0, 1]), frozenset([0, 1, 2])]


In [3]:
lambdas = [[0.02, 0.0199, 0.0198, 0.0198, 0.0197], 
           [0.01, 0.0198, 0.0197, 0.0196, 0.0195]]

print lambdas

[[0.02, 0.0199, 0.0198, 0.0198, 0.0197], [0.01, 0.0198, 0.0197, 0.0196, 0.0195]]


In [4]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

pills = [[1, 2],
         [3, 4],
         [5, 6],
         [5, 6],
         [7, 8]]

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, lambdas, pills)

print "intensities = \n",copula.intensities
print
print "subsets = \n",copula.subsets
print
print "pillars = \n", copula.pillars

intensities = 
[[ 0.02    0.01  ]
 [ 0.0199  0.0198]
 [ 0.0198  0.0197]
 [ 0.0198  0.0196]
 [ 0.0197  0.0195]]

subsets = 
[[frozenset([0])]
 [frozenset([1])]
 [frozenset([2])]
 [frozenset([0, 1])]
 [frozenset([0, 1, 2])]]

pillars = 
[[1 2]
 [3 4]
 [5 6]
 [5 6]
 [7 8]]


In [5]:
for x in copula.subsets:
    print x[0]

frozenset([0])
frozenset([1])
frozenset([2])
frozenset([0, 1])
frozenset([0, 1, 2])


In [6]:
print copula.get_indexes_including(0)
print copula.get_indexes_including(1)
print copula.get_indexes_including(2)

[0, 3, 4]
[1, 3, 4]
[2, 4]


In [7]:
copula.models[0] = None
print copula.models
print

print copula.models[0].pillars
print copula.models[0].intensities
print
print copula.models[1].pillars
print copula.models[1].intensities
print
print copula.models[2].pillars
print copula.models[2].intensities
print
print copula.models[3].pillars
print copula.models[3].intensities
print
print copula.models[4].pillars
print copula.models[4].intensities

[ <credit.default_models.StepwiseConstantIntensity object at 0x0000000012574550>

[0 1 2]
[ 0.02  0.01]

[0 3 4]
[ 0.0199  0.0198]

[0 5 6]
[ 0.0198  0.0197]

[0 5 6]
[ 0.0198  0.0196]

[0 7 8]
[ 0.0197  0.0195]


To generate the default times, we do the following:

\begin{equation*}
\eta_Y = \inf\{ t \geq 0, \int_0^t \gamma_s^Y ds > \xi_Y \}
\end{equation*}

The code uses the `brentq` routine from [scipy](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.brentq.html#scipy.optimize.brentq) to find the default time given the exponential random variable.

In [8]:
from scipy.stats import expon
import numpy as np

#exps = expon.rvs(size=[1, 3])
exps = np.array([[1.0, 2.0, 10000000]])
taus = copula.generate_default_times(obligor_index=0, exp_rvs=exps)

In [9]:
print taus[0, 0], -copula.models[0].log_survival_proba(taus[0, 0]), exps[0, 0]
print taus[0, 1], -copula.models[3].log_survival_proba(taus[0, 1]), exps[0, 1]
print taus[0, 2], -copula.models[4].log_survival_proba(taus[0, 2]), exps[0, 2]

99.0 1.0 1.0
101.989795918 2.0 2.0
1000.0 19.5014 10000000.0


In [10]:
print copula.generate_default_times(1, 10)

[[ 63.20013354   1.48720009   5.8533897 ]]


In [11]:
print copula.generate_default_times(2, 10)

[[ 55.34102106  39.26442294]]


In [12]:
for i in range(7):
    try:
        copula.generate_default_times(i, 20)
    except Exception, e:
        print e

The obligor_index must be lower than 2, given: 3
The obligor_index must be lower than 2, given: 4
The obligor_index must be lower than 2, given: 5
The obligor_index must be lower than 2, given: 6


In [13]:
# Test with 0 intensity, must always be alive!

copula = StepWiseIntensitiesMarshallOlkinCopula([frozenset([1])], [[0.0, 0.0]], [3, 5])

copula.generate_default_times(1, 10)

array([[ 1000.]])

In [15]:
copula.models[0].log_survival_proba(0) == 0

True